# Python test for machine-learning

#### Importing data

In [2]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR
import seaborn as sns
import io
import urllib.request
import matplotlib.pyplot as plt
import os
import collections
import re
import json
import xml.etree.ElementTree as ET
import zipfile
import math

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

chEmbl information was obtained through a select of the database (into mysql in centOS)

In [8]:
data=pd.read_csv('../../DrugDatabasesTools/Chembl/Chembl.csv')
data.head()

C:\Users\joaof\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,molregno,pref_name,chembl_id,max_phase,therapeutic_flag,dosed_ingredient,structure_type,chebi_par_id,molecule_type,first_approval,...,full_mwt,aromatic_rings,heavy_atoms,num_alerts,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations
0,170,NaN,CHEMBL263777,0,0,0,MOL,NaN,Small molecule,NaN,...,588.82,3.0,43.0,1.0,0.16,588.4039,C36H52N4O3,7.0,4.0,1.0
1,184,NaN,CHEMBL6227,0,0,0,MOL,NaN,Small molecule,NaN,...,602.85,3.0,44.0,1.0,0.14,602.4196,C37H54N4O3,7.0,4.0,1.0
2,185,NaN,CHEMBL6242,0,0,0,MOL,NaN,Small molecule,NaN,...,588.82,3.0,43.0,1.0,0.16,588.4039,C36H52N4O3,7.0,4.0,1.0
3,197,NaN,CHEMBL448105,0,0,0,BOTH,NaN,Protein,NaN,...,851.00,1.0,60.0,1.0,0.04,850.4913,C39H66N10O11,21.0,16.0,2.0
4,198,SC-58272,CHEMBL6269,0,0,0,MOL,NaN,Small molecule,NaN,...,596.80,2.0,43.0,1.0,0.16,596.4050,C33H52N6O4,10.0,6.0,1.0


In [9]:
dbdata=pd.read_csv('../../DrugDatabasesTools/DrugBank.ca/drugbank.csv')
dbdata.head()

,drugbank_id,name,type,groups,atc_codes,categories,inchikey,inchi,average-mass,monoisotopic-mass,...,Ghose Filter,MDDR-Like Rule,Drugs Product Database (DPD),PubChem Substance,KEGG Drug,PharmGKB,UniProtKB,Therapeutic Targets Database,ChEMBL,Wikipedia
0,DB00001,Lepirudin,biotech,approved,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",NaN,NaN,NaN,NaN,...,NaN,NaN,11916.0,46507011.0,D06880,PA450195,P01050,DAP000541,CHEMBL1201666,Lepirudin
1,DB00002,Cetuximab,biotech,approved,L01XC06,"Amino Acids, Peptides, and Proteins|Antibodies...",NaN,NaN,NaN,NaN,...,NaN,NaN,13175.0,46507042.0,D03455,PA10040,NaN,DNC000788,CHEMBL1201577,Cetuximab
2,DB00003,Dornase alfa,biotech,approved,R05CB13,"Amino Acids, Peptides, and Proteins|Cough and ...",NaN,NaN,NaN,NaN,...,NaN,NaN,650.0,46507792.0,NaN,PA10318,P24855,DAP000981,CHEMBL1201431,Dornase_alfa
3,DB00004,Denileukin diftitox,biotech,approved|investigational,L01XX29,"ADP Ribose Transferases|Amino Acids, Peptides,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,46506950.0,NaN,PA164750594,P00587,DAP001098,CHEMBL1201550,Denileukin_diftitox
4,DB00005,Etanercept,biotech,approved|investigational,L04AB01,"Amino Acids, Peptides, and Proteins|Analgesics...",NaN,NaN,NaN,NaN,...,NaN,NaN,12032.0,46506732.0,D00742,PA449515,P20333,DNC000605,CHEMBL1201572,Etanercept


to merge the two sources of data

In [ ]:
full=pd.merge(data, dbdata, left_on='chembl_id', right_on='ChEMBL', how='outer', suffixes=('_left', '_right'))
#full[full['ChEMBL'].isnull()==False].head()

In [ ]:
full.dtypes
data=full
data.head(5)

**Evaluating the dataframe created** 
* evaluating datatypes
* evaluating certain columns 
* filtering data
* histogram for numerical values

In [ ]:
#1
data.dtypes
#2
data['published_type'].describe()
#3
data[data["published_type"] == "logP"]
#4
data.hist(column="standard_value")

### para filtrar por count é:

In [ ]:
def evaluate_column(df,column,filter=0):
    if df[column].dtypes=='object':
        eval=df.groupby(column).filter(lambda x: len(x) > filter)
        #plot
        eval[column].value_counts().plot(kind='bar')
        #print list
        eval[column].value_counts().index
        
        info=[#df[column].sum(), # Total sum of the column values
             #df[column].mean(), # Mean of the column values
             #df[column].median(), # Median of the column values
             df[column].nunique(), # Number of unique entries
             #df[column].max(), # Maximum of the column values
             #df[column].min(), # Minimum of the column values
             df[column].isnull().sum(),
        df[column].describe()] 

    else:
        eval=df.hist(column=column)
        info=[df[column].sum(), # Total sum of the column values
             df[column].mean(), # Mean of the column values
             df[column].median(), # Median of the column values
             df[column].nunique(), # Number of unique entries
             df[column].max(), # Maximum of the column values
             df[column].min(), # Minimum of the column values
             df[column].isnull().sum(),
             df[column].describe()] 
        
    return info
evaluate_column(data,'Boiling Point',25)

In [ ]:
#teste purposes
data_eval=data[data["published_type"] == "pKa"]
[len(data_eval)*0.8,
 data_eval['acd_most_apka'].sum(), # Total sum of the column values
 data_eval['acd_most_apka'].mean(), # Mean of the column values
 data_eval['acd_most_apka'].median(), # Median of the column values
 data_eval['acd_most_apka'].nunique(), # Number of unique entries
 data_eval['acd_most_apka'].max(), # Maximum of the column values
 data_eval['acd_most_apka'].min(),
 data_eval['acd_logd'].isna().sum()] # Minimum of the column values

## Cleaning data
#### filtering for pka
pka_data_cleaning from [here](https://www.kaggle.com/mnoori/feature-selection-for-mlr-with-python)


In [ ]:
def data_cleaning(df,null_cutoff,published_type,remove_null='Y'):
    
    #removing duplicate and unmeaninfull columns
    df=df.drop(columns=["md", "cp","cr","at","aa","molregno.2","molregno.1","doc_id.1","doc_id.2","molregno.3","doc_id.1","doc_id.2"])
    df=df.drop(columns=["src_id.1","chembl_id.1","assay_id.1","record_id.1"])
    
    #selecting for pka
    df=df[df["published_type"] == published_type]

    #keeping interesting columns
    df=df[["max_phase","dosed_ingredient", "structure_type",  "molecule_type",
    "oral", "parenteral", "topical", "black_box_warning",
    "natural_product", "first_in_class", "chirality", "prodrug",
    "inorganic_flag", "usan_year", "availability_type", "usan_stem",
    "polymer_flag", "usan_substem", "usan_stem_definition",
    "indication_class", "withdrawn_flag", "withdrawn_year",
    "withdrawn_country", "withdrawn_reason", "mw_freebase","alogp","hba",
    "hbd", "psa", "rtb", "ro3_pass", "num_ro5_violations", "acd_most_apka",
    "acd_most_bpka", "acd_logp", "acd_logd", "molecular_species",
    "full_mwt", "aromatic_rings", "heavy_atoms", "num_alerts",
    "qed_weighted", "mw_monoisotopic",  "hba_lipinski",
    "hbd_lipinski", "num_lipinski_ro5_violations","assay_type", "relationship_type",
    "confidence_score","standard_value","half-life","toxicity","metabolism","absorption",
    "smiles","melting point","logS EXP","logP EXP","pKa EXP","Isoelectric Point",
    "Molecular Weight","Molecular Formula","Hydrophobicity","Boiling Point","Caco2 Permeability",
    "Water Solubility EXP","PSA calc","Refractivity calc","Polarizability","Ghose Filter","MDDR-Like Rule",
    "average-mass","monoisotopic-mass", "volume-of-distribution","clearance"]]
    

    #removing outlier far greater than average
    if published_type in ["pKa"]:
        df=df[df["standard_value"]<400]
    
    if remove_null=='N':
        null_cutoff=0
        
    #dropping columns with more than a missing values
    #null_values=df.isnull().sum()
    #drop_missing_values=null_values[null_values>(null_cutoff*len(df))]
    #df=df.drop(drop_missing_values.index, axis=1)    
    df = df.dropna(thresh=len(df)-null_cutoff*len(df), axis=1)
    
    # counting null values in text columns
    text_cols_nullcount=df.select_dtypes(include=["object"]).isnull().sum().sort_values(ascending=False)
    text_cols_nullcols=text_cols_nullcount.index
    for col in text_cols_nullcols:
        mostcounts=df[col].value_counts().index.tolist()
        df[col]=df[col].fillna(mostcounts[0]) #replacing the missing column in a text with the highest number of values

    #missing values in numerical columns 
    num_cols=df.select_dtypes(include=["object","float64"]).columns #selecting numerical columns
    num_null_counts=df[num_cols].isnull().sum().sort_values(ascending=False) #counting null values in columns
    num_null_cols=num_null_counts[num_null_counts!=0].index #selecting the ones that have missing values
    df=df.fillna(df[num_null_cols].mode().to_dict(orient="records")[0]) #replacing missing with mode
    #passing categorical to numerical
    df=pd.get_dummies(df, prefix="is_")#add column to name as well

    #remove duplicates
    df=df.drop_duplicates()
    
    return df

In [ ]:
pka_data=data_cleaning(data,0.8,'pKa',remove_null='N')
pka_data

In [ ]:
pka_data.dtypes

In [ ]:
logp_data=data_cleaning(data,0.8,'logP')
logp_data

analisado o gráfico em baixo, vemos que não existe correlação forte de de nenhuma coluna com o alvo - **standard_value**

In [ ]:
logd_data=data_cleaning(df=data,published_type='logD',null_cutoff=0.8)    
logd_data

In [ ]:
def check_correlation(df,target,corr_cutoff):
    data_train=df.sample(frac=0.7,random_state=200)
    data_test=df.drop(data_train.index)

    data_x=df.drop(columns=[target])
    data_y=df[target]

    data_x_train=data_train.drop(columns=[target])
    data_y_train=data_train[target]

    data_x_test=data_test.drop(columns=[target])
    data_y_test=data_test[target]
    
    corr=data_train.corr()
    #fig,ax=plt.subplots(figsize=(8,6))
    #sns.heatmap(corr)
    features=''
    features_text=''
    if len(corr[target].where(lambda x : x.abs()>corr_cutoff).dropna())>1:
        features=corr[target].where(lambda x : x.abs()>corr_cutoff).dropna()
        features_text=features.index.str.cat(sep=', ')+'\n'
    else:
        features='1'
        features_text='None'
    #print('The features correlated with target above the threshold %s are %s' %(corr_cutoff,features_text))
    return len(features)

check_correlation(pka_data,'standard_value',0.01)

para retirar colunas com variância abaixo de X, mas devolve um np  
além disso, temos variaveis "booleanas" o que torna complicado aplicar isto pq a variância não há de ser muito grande

In [ ]:
#sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
#sel.fit_transform(pka_data_corrected_1)

In [ ]:
def xMlr(df,target,frac=0.7,cv=10):
    i=0
    mse=0
    score=0
    while i<cv:
        np.random.seed(seed=123)
        pka_data_train=df.sample(frac=0.7,random_state=200)
        pka_data_test=df.drop(pka_data_train.index)

        pka_data_x=df.drop(columns=[target])
        pka_data_y=df[target]

        pka_data_x_train=df.drop(columns=[target])
        pka_data_y_train=df[target]

        pka_data_x_test=df.drop(columns=[target])
        pka_data_y_test=df[target]
        regr = linear_model.LinearRegression()
        regr.fit(pka_data_x_train, pka_data_y_train)
        #print(regr.coef_)
        mse+=(np.mean((regr.predict( pka_data_x_test)-pka_data_y_test)**2))
        score+=regr.score(pka_data_x_test, pka_data_y_test)
        i+=1
    return mse/cv, score/cv


MSE is mean squared error  
Explained variance score:   
1 is perfect prediction
and 0 means that there is no linear relationship
between X and y.

In [ ]:
mlr_mse,mlr_score=xMlr(pka_data,'standard_value')
print("RMSE is %s. Score is %s." % (mlr_mse, mlr_score))

# SVR

In [ ]:
def xSVR(df,target,frac=0.7,cv=10):
    i=0
    mse=0
    score=0
    while i<cv:
        np.random.seed(seed=123)
        pka_data_train=df.sample(frac=0.7,random_state=200)
        pka_data_test=df.drop(pka_data_train.index)

        pka_data_x=df.drop(columns=[target])
        pka_data_y=df[target]

        pka_data_x_train=df.drop(columns=[target])
        pka_data_y_train=df[target]

        pka_data_x_test=df.drop(columns=[target])
        pka_data_y_test=df[target]
        clf = SVR(gamma='scale', C=1.0, epsilon=0.1)
        clf.fit(pka_data_x_train, pka_data_y_train) 
        mse+=(np.mean((clf.predict( pka_data_x_test)-pka_data_y_test)**2))
        score+=clf.score(pka_data_x_test, pka_data_y_test, sample_weight=None)
        i+=1
    return mse/cv, score/cv

In [ ]:
svr_mse,svr_score=xSVR(pka_data,'standard_value')
print("MSE is %s. Score is %s." % (svr_mse, svr_score))

In [ ]:
def evaluation_test_train(df,length,cv,null_cutoff,correlation):
    result={}
    eval_df=df.groupby("published_type").filter(lambda x: len(x) > length)
    test_list=eval_df["published_type"].value_counts().index
    
    for item in test_list:
        try:
            test_df=data_cleaning(df,null_cutoff,str(item))
            if check_correlation(corr_cutoff=correlation,df=test_df,target='standard_value')>1:
                result[item]=xMlr(test_df,'standard_value',frac=0.7,cv=cv)
        except:
            print(item)
            continue
    return result

data_result=evaluation_test_train(data,1000,5,0.8,0.1)
print (data_result)